# structures and definitions

In [1]:
# lets user simple arrays of length 6 to represent both borrow/deposit positions, as well as scalers

# the following are the indices 
LB = 1
XB = 2
YB = 3
LD = 4
XD = 5
YD = 6

# our main dimension is r = sqrt of price. more specifically, we will main use rX = the sqrt of the price in X of Y

pX = 4
r = rX = √pX

L̄ = LTVMAX = 0.75

0.75

# conversions

In [2]:
sharesToAssets(position, scalers) = position .* scalers
convertUnitsXYToL(rX, position) = 
[
    position[LB],
    convertUnitsXToL(rX, position[XB]),
    convertUnitsYToL(rX, position[YB]),
    position[LD],
    convertUnitsXToL(rX, position[XD]),
    convertUnitsYToL(rX, position[YD]),
]
convertUnitsXToL(rX, x) = x / 2rX # == x / (2rX) != x / 2*rX == (x / 2)*rX see https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#numeric-literals
convertUnitsYToL(rX, y) = y * rX / 2

convertUnitsYToL (generic function with 1 method)

# saturation functions

In [3]:
function calcNetBorrowXAndNetBorrowY(rX, positionInNativeShares, scalers)
    # convert position to units of liquidity assets
    positionInNativeAssets = sharesToAssets(positionInNativeShares, scalers)
    positionInLAssets = convertUnitsXYToL(rX, positionInNativeAssets)
    
    borrowX = positionInLAssets[LB] / 2 + positionInLAssets[XB]
    depositX = positionInLAssets[LD] / 2 + positionInLAssets[XD]
    borrowY = positionInLAssets[LB] / 2 + positionInLAssets[YB]
    depositY = positionInLAssets[LD] / 2 + positionInLAssets[YD]

    netBorrowX = borrowX - depositX
    netBorrowY = borrowY - depositY
    
    netBorrowX, netBorrowY
end

function calcLTV(rX, positionInNativeShares, scalers, L̄)
    netBorrowX, netBorrowY = calcNetBorrowXAndNetBorrowY(rX, positionInNativeShares, scalers)
    borrow = max(netBorrowX, netBorrowY)
    
    isNetX = netBorrowX == borrow

    0 < netBorrowX && 0 < netBorrowY && return L̄, isNetX # all borrow => LTV = inf <=> LTV == L̄

    netBorrowX <= 0 && netBorrowY <= 0 && return 0, isNetX # all deposit => LTV = 0

    deposit = -min(netBorrowX, netBorrowY)

    ltv = borrow / deposit

    L̄ ≤ ltv && return L̄, isNetX
    
    ltv, isNetX
end



calcLTV (generic function with 1 method)

# parabola

In [4]:
# from the condition LTV == L̄, we can derive a parabola with r as the variable:
# a * rX^2 + b * rX + c == 0

function calcNetBorrowAssetsLXY(positionInNativeShares, scalers)
    # convert to native assets
    positionInNativeAssets = sharesToAssets(positionInNativeShares, scalers)

    L̂ = positionInNativeAssets[LB] - positionInNativeAssets[LD]
    X̂ = positionInNativeAssets[XB] - positionInNativeAssets[XD]
    Ŷ = positionInNativeAssets[YB] - positionInNativeAssets[YD]

    L̂, X̂, Ŷ
end

function calcParabolaParamsAssumingNetX(rX, positionInNativeShares, scalers, L̄) # returns a, b, c
    L̂, X̂, Ŷ = calcNetBorrowAssetsLXY(positionInNativeShares, scalers)
    L̄ * Ŷ, (L̄ - 1) * L̂, -X̂
end

function calcParabolaParamsAssumingNetY(rX, positionInNativeShares, scalers, L̄) # returns a, b, c
    L̂, X̂, Ŷ = calcNetBorrowAssetsLXY(positionInNativeShares, scalers)
    Ŷ, (1 - L̄) * L̂, -L̄ * X̂
end

function calcQuadraticSolutions(a, b, c) # returns rXPlus, rXMinus
    a == 0 && return -c / b, 0
    c == 0 && return 0, -b / a
    b == 0 && return √(-c/a), -√(-c/a)

    radical = √(b^2 - 4*a*c)

    (-b + radical) / 2a, (-b - radical) / 2a
end

calcQuadraticSolutions (generic function with 1 method)

# graphing

In [8]:
using WGLMakie
fig = Figure()

r_range = 0.01:0.01:10
lxy_range = 0:0.1:20
startValues = zeros(6)

# create sliders
sl_lxy = SliderGrid(
    fig[1, 1], 
    (label = "LB", range = lxy_range, format = "{:.2f} [LS]", startvalue = startValues[LB]),
    (label = "XB", range = lxy_range, format = "{:.2f} [XS]", startvalue = startValues[XB]),
    (label = "YB", range = lxy_range, format = "{:.2f} [YS]", startvalue = startValues[YB]),
    (label = "LD", range = lxy_range, format = "{:.2f} [LS]", startvalue = startValues[LD]),
    (label = "XD", range = lxy_range, format = "{:.2f} [XS]", startvalue = startValues[XD]),
    (label = "YD", range = lxy_range, format = "{:.2f} [YS]", startvalue = startValues[YD]),
)

# create graph axis
ax = Axis(fig[2, 1], limits = ((0, maximum(r_range)), (-1, 1)), xticks = 0:0.1:maximum(r_range), yticks = -1:0.25:1)

# LTVMAX = 0.75
hlines!(ax, L̄, color = :yellow)

Plot{Makie.hlines, Tuple{Float64}}